# Ecosystem emissivity, Summer data (Following Thakur et al. 2022)

Thakur, G., Schymanski, S.J., Mallick, K. et al. Downwelling longwave radiation and sensible heat flux observations are critical for surface temperature and emissivity estimation from flux tower data. Sci Rep 12, 8592 (2022). https://doi.org/10.1038/s41598-022-12304-3

In [ ]:
import pandas as pd
import numpy as np
import glob
import scipy.stats
from plotnine import *

## Functions

In [ ]:
# Stefan-Boltzmann constant
sigma = 5.670374419 * 10**(-8) # [W m−2 K−4]

In [ ]:
def load_data(fn, index_col=False, silent=False):
    if (not silent):
        print('-', fn.split('/')[-1])
    temp = pd.read_csv(fn, index_col=index_col)
    temp['DateTime'] = pd.to_datetime(temp['DateTime'], format='%Y-%m-%d %H:%M:%S', utc=True)
    return(temp)

# Function to calculate RMSE
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

### Method by Juang et al. (2007)

This method has been used by extensively by Novick & Katul (e.g. 2020), and their co-authors, but it has major problems

- Empirical equation: $\varepsilon = -0.16 \alpha + 0.99$
- They claim that $R^2 = 0.94$ in their creation of this equation, but upon re-analysis by Mila Volkov (MSc student), we found no significant correlation at all

In [ ]:
def emissivity_juang2007(in_df):
    temp = in_df.copy()
    # Calculate albedo
    #temp['albedo'] = temp['SWout'] / temp['SWin']
    # Keep only daytime values with large Rn
    temp = temp.loc[temp['Rn'] > 100]
    # Calculate emissivity for each datapoint
    temp['emissivity'] = -0.16 * temp['albedo'] + 0.99
    # calculate mean & standard deviation
    emissivity = np.mean(temp['emissivity'])
    emissivity_sd = np.std(temp['emissivity'])
    return(emissivity, emissivity_sd)

### Method by Maes et. al (2019)

- Developed an equation of the form: $\varepsilon = \frac{R_{l,\uparrow} - R_{l,\downarrow}}{T_a^4 \sigma - R_{l,\downarrow}}$
- For data filtered according to $−2 < H < 2$ , and $\alpha < 0.4$

In [ ]:
def emissivity_maesetal2019(in_df):
    # Stefan-Boltzmann constant
    sigma = 5.670374419 * 10**(-8) # [W m−2 K−4]
    # Filter according to Maes et al. (2019)
    temp = in_df.loc[(in_df['H'] > -2) & (in_df['H'] < 2) & ((in_df['SWout']/in_df['SWin']) > 0.4)].copy()
    # Calculate emissivity for each datapoint, using air temperature in Kelvin
    temp['emissivity'] = (temp['Lout'] - temp['Lin']) / (temp['Ta']**4 * sigma - temp['Lin'])
    # calculate mean & standard deviation
    emissivity = np.median(temp['emissivity'])
    emissivity_sd = np.std(temp['emissivity'])
    return(emissivity, emissivity_sd)

### Method by Thakur et al. (2022): BEST METHOD

We start with the equations for radiometers:
- Short eq.: $R_{l,\uparrow} \approx \varepsilon\sigma T^4$
- Long eq., ignoring the air column: $R_{l,\uparrow} = \varepsilon\sigma T^4_s + (1 - \varepsilon) R_{l,\downarrow}$
- Solving the long eq. for $T_s$ yields: $ T_s = \sqrt[4]{\frac{R_{l,\uparrow} - (1 - \varepsilon) R_{l,\downarrow}}{\varepsilon\sigma}} = \sqrt[4]{\frac{R_{l,\uparrow}}{\varepsilon\sigma} - \frac{R_{l,\downarrow}}{\varepsilon\sigma} + \frac{R_{l,\downarrow}}{\sigma} }$ (i.e. Thakur et al., 2021; eq. 7)

The sensible heat equations are the following:
- Sensible heat equation: $H = \rho c_p \frac{T_s - T_a}{r_H}$
- Alternatively, base equation: $H = m(Ts − Ta) + c$, where $m$ is the proportionality constant or heat transfer coefficient, and $c$ is the offset due to the $H$ from surfaces in the aerodynamic footprint that are not seen by the radiometer

Data should be filtered slightly:
- Wind speed larger than 0.1 m s-1
- Net radiation larger than 25 W m-2

In [ ]:
# Search for ecosystem emissivity
def search_emissivity(in_df):
    # Stefan-Boltzmann constant
    sigma = 5.670374419 * 10**(-8) # [W m−2 K−4]
    # Create a df to store all the tested values
    results = pd.DataFrame(columns=["RMSE", "m","eps_ref","R2","c"])
    # Create a list of potential emissivities
    emissivity_range = np.arange(0.7,1,0.001).tolist() # using array of the emissivity values
    # Filter the input data
    temp = in_df.loc[in_df['Rn'] > 25].copy()
    
    # Function to calculate RMSE
    def rmse(predictions, targets):
        return np.sqrt(((predictions - targets) ** 2).mean())
    
    # Prepare empty data
    data = []
    # Calculate values for all emissivities in the range
    for emissivity in emissivity_range:
        emissivity = np.round(emissivity, 3)
        # Surface temperature [K]
        temp['Ts'] = ((temp['Lout'] - (1 - emissivity) * temp['Lin']) / (emissivity * sigma))**(1/4)
        # Surface-Air temperature difference (Note: Ta must be in K)
        temp['dT'] = temp['Ts'] - temp['Ta']
        # Eliminate data where either H or dT is NA
        temp = temp.loc[~temp['dT'].isna() & ~temp['H'].isna()].copy()
        #display(temp[['dT', 'H']])
        
        # Set x and y
        dT = temp['dT']
        H = temp['H']
        # Try a linear regression (Git's method)
        try:
            slope, intercept, r2, p, stderr = scipy.stats.linregress(dT, H)
        except:
            # If the regression didn't work, R2 should be 0 (no correlation)
            r2=0
        else:
            RMSE = rmse(intercept + slope * dT, H)
        
        # Add to resulting data list
        #data.append([emissivity, slope, intercept, r2, p, stderr, RMSE]) # Appends everything, even bad data
        #print(emissivity, slope, intercept, r2, p, stderr, RMSE)
        if r2 > 0.5:
            data.append([emissivity, slope, intercept, r2, p, stderr, RMSE])
        else:
            data.append([emissivity, np.nan, np.nan,   r2, np.nan, np.nan, np.nan])
        
        # Convert to dataframe
        out_df = pd.DataFrame(data, columns=['emissivity', 'slope', 'intercept', 'R2', 'P', 'stderr', 'RMSE'])
        
        # Sort by RMSE and R2
        out_df.sort_values(by=['RMSE', 'R2'], ascending=True, inplace=True)
    return(out_df)

## Load data

In [ ]:
# Data location
project_path = './'

# Input
data_path = project_path + '../../data/'
graphs_path = project_path + '../graphs/'

# Data files
yatir_desert_fn = data_path + 'dataset.csv'

In [ ]:
print("Loading data")
print("------------")

df = load_data(yatir_desert_fn)
# Selecting Afforestation Desert Background campaigns
df = df.loc[df['Ecosystem'] == 'Afforestation desert background'].copy()

df['Rn'] = df['SWin'] - df['SWout'] + df['Lin'] - df['Lout']

df['Ta'] = df['Ta'] + 273.15 # Convert from °C to K

print('Done...')

In [ ]:
df_summer1 = df.loc[(df['Month'] != 'March') & (df['Year'] == 2013)]
df_summer2 = df.loc[(df['Month'] != 'March') & (df['Year'] == 2015)]

#### Check correlation between sonic and air temperature

In [ ]:
temp = df_summer1.copy()
temp = temp.loc[~temp['Ta'].isna() & ~temp['Tsonic'].isna()].copy()

# Do the regression
slope, intercept, r2, p, stderr = scipy.stats.linregress(temp['Tsonic'], temp['Ta'])
# Show results
print('T air sonic vs. normal')
print('Slope:     ', slope)
print('Intercept: ', intercept)
print('R2:        ', r2)
print('P-value:   ', p)
print('Std. Err.: ', stderr)

# Plot
plt = ggplot(temp)
plt = plt + geom_point(aes(x='Tsonic', y='Ta'), size=1)
plt = plt + labs(x='$T_{a}$ (Sonic) [K]', y='$T_{a}$ (Normal) [K]', parse=True)
plt = plt + geom_smooth(aes(x='Tsonic', y='Ta'), method='lm', size=1)
plt = plt + theme_bw()
plt = plt + theme(text=element_text(family="serif"))
print(plt)

## Afforestation Desert Background (Summer 2013 campaign)

### Method by Juang et al. (2007)

- Empirical equation: $\varepsilon = -0.16 \alpha + 0.99$

In [ ]:
emissivity_juang = emissivity_juang2007(df_summer1)
emissivity_juang2007(df_summer1)

#### Method by Maes et. al (2019)

- Equation: $\varepsilon = \frac{R_{l,\uparrow} - R_{l,\downarrow}}{T_a^4 \sigma - R_{l,\downarrow}}$
- Data filter: $−2 < H < 2$ , and $\alpha < 0.4$

In [ ]:
emissivity_maes = emissivity_maesetal2019(df_summer1)
print('Emissivity (Maes et al. 2019):', np.round(emissivity_maes[0], 2), '±' , np.round(emissivity_maes[1], 2))

#### Method by Thakur et al. (2022): BEST METHOD

- Long eq. for thermal radiation (ignoring the air column) solved for $T_s$ yields: $ T_s = \sqrt[4]{\frac{R_{l,\uparrow} - (1 - \varepsilon) R_{l,\downarrow}}{\varepsilon\sigma}}$ (i.e. Thakur et al., 2021; eq. 7)
- Find correlation between $H$ and $Ts − Ta$ using: $H = m(Ts − Ta) + c$, where $m$ is the proportionality constant or heat transfer coefficient, and $c$ is the offset due to the $H$ from surfaces in the aerodynamic footprint that are not seen by the radiometer
- Data filter: Wind speed larger than 0.1 m s-1, Net radiation larger than 25 W m-2

Results:
- Criteria for choice of a certain  emissivity:
  - Intercept should be < 5-10% of $H$ (here, ca. $15-30 W m^{-2}$
  - Good agreement between Thakur et al. (2022) & Maes et al. (2019) gives us more confidence
  - The resulting slope is consistent to getting an $r_H \approx 100 s m^{-1}$

In [ ]:
# Search emissivity
out_df = search_emissivity(df_summer1)

# Show some results
chosen_emissivity = np.round(emissivity_maes[0],3)
display(out_df.loc[(out_df['emissivity'] > (chosen_emissivity - 0.003)) & (out_df['emissivity'] < (chosen_emissivity + 0.003))])

In [ ]:
# Chosen value
# This is a footprint mismatch of ca 30 W m-2 (10%), considered acceptable by Thakur et al. (2022)
print('Chosen emissivity:', out_df.loc[out_df['emissivity'] == chosen_emissivity, 'emissivity'].values[0])

print('')
print('Statistical information:')
display(out_df.loc[out_df['emissivity'] == chosen_emissivity])

#### Plot results

In [ ]:
# Show RMSE curve
plt = ggplot(out_df)
plt = plt + geom_point(aes(x='emissivity', y='RMSE', colour='intercept'), size=1)
plt = plt + labs(x='emissivity', y='RMSE', colour='Intercept', parse=True)
plt = plt + theme_bw()
plt = plt + theme(text=element_text(family="serif"))
print(plt)

In [ ]:
temp = df.loc[df['Rn'] > 25].copy()

# Emissivity
emissivity = chosen_emissivity

# Surface temperature [K]
temp['Ts'] = ((temp['Lout'] - (1 - emissivity) * temp['Lin']) / (emissivity * sigma))**(1/4)
# Surface-Air temperature difference (Note: Ta must be in K)
temp['dT'] = temp['Ts'] - temp['Ta']
# Filter
temp = temp.loc[~temp['dT'].isna() & ~temp['H'].isna()].copy()
temp['time'] = temp['DateTime'].dt.strftime('%H')

slope, intercept, r2, p, stderr = scipy.stats.linregress(temp['dT'], temp['H'])
RMSE = rmse(intercept + slope * temp['dT'], temp['H'])

print('Emissivity:', emissivity)
print('Slope:     ', slope)
print('Intercept: ', intercept)
print('R2:        ', r2)
print('P-value:   ', p)
print('Std. Err.: ', stderr)
print('RMSE:      ', RMSE)

plt = ggplot(temp)
plt = plt + geom_point(aes(x='dT', y='H', colour='time'), size=1)
plt = plt + labs(x='$\Delta T_{S-A}$ [°C]', y='$H$ [$W~m^{-2}$]', colour='Hour', parse=True)
plt = plt + theme_bw()
plt = plt + theme(text=element_text(family="serif"))
print(plt)

In [ ]:
emissivity_summer1 = emissivity

## Afforestation Desert Background (Summer 2015 campaign)

### Method by Juang et al. (2007)

- Empirical equation: $\varepsilon = -0.16 \alpha + 0.99$

In [ ]:
emissivity_juang2007(df_summer2)

#### Method by Maes et. al (2019)

- Equation: $\varepsilon = \frac{R_{l,\uparrow} - R_{l,\downarrow}}{T_a^4 \sigma - R_{l,\downarrow}}$
- Data filter: $−2 < H < 2$ , and $\alpha < 0.4$

In [ ]:
emissivity_maes = emissivity_maesetal2019(df_summer2)
print('Emissivity (Maes et al. 2019):', np.round(emissivity_maes[0], 2), '±' , np.round(emissivity_maes[1], 2))

#### Method by Thakur et al. (2022): BEST METHOD

- Long eq. for thermal radiation (ignoring the air column) solved for $T_s$ yields: $ T_s = \sqrt[4]{\frac{R_{l,\uparrow} - (1 - \varepsilon) R_{l,\downarrow}}{\varepsilon\sigma}}$ (i.e. Thakur et al., 2021; eq. 7)
- Find correlation between $H$ and $Ts − Ta$ using: $H = m(Ts − Ta) + c$, where $m$ is the proportionality constant or heat transfer coefficient, and $c$ is the offset due to the $H$ from surfaces in the aerodynamic footprint that are not seen by the radiometer
- Data filter: Wind speed larger than 0.1 m s-1, Net radiation larger than 25 W m-2

In [ ]:
# Search emissivity
out_df = search_emissivity(df_summer2)

Results:
- Criteria for choice of a certain  emissivity:
  - Intercept should be < 5-10% of $H$ (here, ca. $15-30 W m^{-2}$
  - Here, Maes et al. (2019) yields an emissivitiy greater than 1, so only the Thakur et al. (2022) method can be considered
  - Assuming no footprint mismatch (desert site), we choose the value with the smallest absolute intercept 

In [ ]:
# Chose value with smallest absolute intercept
out_df['abs_intercept'] = np.abs(out_df['intercept'])
out_df.sort_values(by=['abs_intercept'], ascending=True, inplace=True)
out_df.drop('abs_intercept', axis=1, inplace=True)

# Save chosen emissivity
chosen_emissivity = out_df['emissivity'].values[0]

# Chosen value
# This is a footprint mismatch of ca 30 W m-2 (10%), considered acceptable by Thakur et al. (2022)
print('Chosen emissivity:', out_df.loc[out_df['emissivity'] == chosen_emissivity, 'emissivity'].values[0])

print('')
print('Statistical information:')
display(out_df.loc[out_df['emissivity'] == chosen_emissivity])

#### Plot results

In [ ]:
# Show RMSE curve
plt = ggplot(out_df)
plt = plt + geom_point(aes(x='emissivity', y='RMSE', colour='intercept'), size=1)
plt = plt + labs(x='emissivity', y='RMSE', colour='Intercept', parse=True)
plt = plt + theme_bw()
plt = plt + theme(text=element_text(family="serif"))
print(plt)

In [ ]:
# Show RMSE curve
plt = ggplot(out_df)
plt = plt + geom_point(aes(x='emissivity', y='R2', colour='intercept'), size=1)
plt = plt + labs(x='emissivity', y='R2', colour='Intercept', parse=True)
plt = plt + theme_bw()
plt = plt + theme(text=element_text(family="serif"))
print(plt)

In [ ]:
temp = df_summer2.loc[df['Rn'] > 25].copy()

# Emissivity
emissivity = chosen_emissivity

# Surface temperature [K]
temp['Ts'] = ((temp['Lout'] - (1 - emissivity) * temp['Lin']) / (emissivity * sigma))**(1/4)
# Surface-Air temperature difference (Note: Ta must be in K)
temp['dT'] = temp['Ts'] - temp['Ta']
# Filter
temp = temp.loc[~temp['dT'].isna() & ~temp['H'].isna()].copy()
temp['time'] = temp['DateTime'].dt.strftime('%H')

slope, intercept, r2, p, stderr = scipy.stats.linregress(temp['dT'], temp['H'])
RMSE = rmse(intercept + slope * temp['dT'], temp['H'])

print('Emissivity:', emissivity)
print('Slope:     ', slope)
print('Intercept: ', intercept)
print('R2:        ', r2)
print('P-value:   ', p)
print('Std. Err.: ', stderr)
print('RMSE:      ', RMSE)

plt = ggplot(temp)
plt = plt + geom_point(aes(x='dT', y='H', colour='time'), size=1)
plt = plt + labs(x='$\Delta T_{S-A}$ [°C]', y='$H$ [$W~m^{-2}$]', colour='Hour', parse=True)
plt = plt + theme_bw()
plt = plt + theme(text=element_text(family="serif"))
print(plt)

In [ ]:
emissivity_summer2 = emissivity

## Mean of 2 seasons

In [ ]:
print('Summer 2013:', emissivity_summer1)
print('Summer 2015:', emissivity_summer2)
print('Mean:', np.round(np.mean([emissivity_summer1, emissivity_summer2]),2))